In [ ]:
# ==================================================================================
# Pandas를 이용한 CSV 파일 로드 및 기본 정보 확인 스크립트
# ==================================================================================
#
# [ 주요 기능 ]
# 1. CSV 파일 로드: Pandas 라이브러리의 `read_csv` 함수를 사용하여 지정된 CSV 파일을
#    데이터프레임(DataFrame) 형태로 불러옵니다.
#    - `engine="python"` 옵션은 파일 경로에 한글이 포함되거나 파싱 오류가 발생할 때
#      안정적으로 파일을 읽도록 돕습니다.
# 2. 데이터 구조 확인: `df.info()` 메서드를 호출하여 데이터프레임의 전체적인 정보를
#    출력합니다. (전체 행 개수, 각 열의 데이터 타입, 누락되지 않은 값의 개수 등)
# 3. 데이터 미리보기: `df.tail()` 메서드를 사용하여 데이터의 마지막 5개 행을 출력,
#    데이터가 어떻게 구성되어 있는지 빠르게 확인합니다.
# 4. 오류 처리: `try-except` 구문을 사용하여 파일을 찾을 수 없는 경우(FileNotFoundError)나
#    기타 예외가 발생했을 때, 프로그램이 중단되지 않고 적절한 오류 메시지를
#    출력하도록 처리합니다.
#
# [ 사용 방법 ]
# 1. `file_name` 변수에 읽어올 CSV 파일의 이름을 정확히 입력합니다.
# 2. 이 스크립트를 CSV 파일과 동일한 폴더(디렉토리)에 위치시키거나,
#    파일 경로를 정확하게 지정합니다.
# 3. 스크립트를 실행하면 터미널에 파일 로드 결과와 데이터 정보가 출력됩니다.
#
# ==================================================================================


import pandas as pd


file_name = "github_dataset.csv"


print(f"원본 파일 '{file_name}'을 불러옵니다...")


try:

    df = pd.read_csv(file_name, engine="python")


    print("\n✅ 원본 파일 로드 성공!")

    print("데이터프레임 정보:")

    df.info()


    print(f"\n전체 행 개수: {len(df)}")

    print("\n데이터 미리보기:")

    print(df.tail())


except FileNotFoundError:
    print(

        f"오류: '{file_name}' 파일을 찾을 수 없습니다. 파일이 코드와 같은 폴더에 있는지 확인해주세요."
    )

except Exception as e:

    print(f"파일을 읽는 중 오류가 발생했습니다: {e}")

원본 파일 'github_dataset.csv'을 불러옵니다...

✅ 원본 파일 로드 성공!
데이터프레임 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2927 entries, 0 to 2926
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          2927 non-null   int64  
 1   transcript  2927 non-null   object 
 2   confidence  0 non-null      float64
 3   label       2927 non-null   int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 91.6+ KB

전체 행 개수: 2927

데이터 미리보기:
       id                                         transcript  confidence  \
2922  690  지금까지 사실확인서를 연락드렸습니다. 하고 있는데 이런 사건 때문에 내가 여쭤볼게 ...         NaN   
2923  691  연료 데이는 사건 때문에 연락 드렸는데 혹시 김명철씨 가십니까 모르시고 아는 사람도...         NaN   
2924  692  4장 4절 일어났습니까? 장소 잡고 님께서는 42세 남성 김동술 알고 계십니까? 아...         NaN   
2925  693  네\n네 여보세요\n뭐 어디세요?\n아 네 안녕하세요 저희 그 하나캐피탈에서 연락을...         NaN   
2926  694  통장명이 (삐-) 계좌 맞구요? 그리고 통장 발급 일은 2017년 3월 14일입니다...         NaN   

      label  
2922      1  
2923      

In [67]:
df.iloc[2231]

id                                                         2231
transcript    저는 학교를 졸업한 지 오래되어서 어~ 기억이 쪼금 먼 기리 멀기는 하지만 어~ 저...
confidence                                                  NaN
label                                                         0
Name: 2231, dtype: object

In [ ]:
# ==================================================================================
# Pandas를 이용한 데이터셋 분리 및 전처리 스크립트
# ==================================================================================
#
# [ 주요 기능 ]
# 1. 데이터 분리: 원본 CSV 파일('github_dataset.csv')을 불러온 후, 미리 지정된
#    행 인덱스(PHISHING_ROW_INDICES)를 기준으로 '피싱(phishing)' 데이터와
#    '일반(normal)' 데이터를 두 개의 데이터프레임으로 분리합니다.
#
# 2. 열 이름 변경 및 정제:
#    - '피싱' 데이터는 'file_name', 'phishing_type', 'speaker', 'text' 열을 선택하고,
#      그에 맞게 열 이름을 표준화합니다.
#    - '일반' 데이터는 'file_name', 'text' 열만 선택하고, 동일하게 열 이름을 맞춥니다.
#
# 3. 파일 저장: 전처리가 완료된 두 개의 데이터프레임을 각각 'phishing_data.csv'와
#    'normal_data.csv'라는 새로운 파일로 저장합니다.
#    - `encoding="utf-8-sig"` 옵션을 사용하여 한글 깨짐 문제를 방지합니다.
#
# 4. 강력한 오류 처리: 파일이 없거나, 지정한 행 번호가 범위를 벗어나거나,
#    특정 열 이름이 존재하지 않는 등 다양한 예외 상황에 대한 오류 메시지를 출력하여
#    문제 해결을 돕습니다.
#
# [ 사용 방법 ]
# 1. `FILE_TO_READ` 변수에 원본 CSV 파일명을 입력합니다.
# 2. `PHISHING_ROW_INDICES` 리스트에 피싱 데이터에 해당하는 행 번호 범위를 지정합니다.
# 3. 스크립트를 실행하면, 동일한 폴더에 결과물인 csv 파일 2개가 생성됩니다.
#
# ==================================================================================


import pandas as pd

import os



FILE_TO_READ = "github_dataset.csv"


PHISHING_ROW_INDICES = list(range(2232, 2927))



try:


    current_dir = os.getcwd()


    file_path = os.path.join(current_dir, FILE_TO_READ)



    print(f"파일을 다음 경로에서 찾습니다: {file_path}")



    df = pd.read_csv(file_path, engine='python')


    print(f"\n✅ '{FILE_TO_READ}' 파일을 성공적으로 불러왔습니다. (총 {len(df)}개 행)")



    phishing_df = df.iloc[PHISHING_ROW_INDICES].copy()


    normal_df = df[~df.index.isin(PHISHING_ROW_INDICES)].copy()



    phishing_final_df = phishing_df.rename(columns={


        'id': 'file_name',


        'confidence': 'phishing_type',


        'label': 'speaker',


        'transcript': 'text'


    })



    phishing_final_df = phishing_final_df[['file_name', 'phishing_type', 'speaker', 'text']]




    normal_final_df = normal_df.rename(columns={


        'id': 'file_name',


        'transcript': 'text'


    })



    normal_final_df = normal_final_df[['file_name', 'text']]



    phishing_final_df.to_csv("phishing_data.csv", index=False, encoding="utf-8-sig")


    normal_final_df.to_csv("normal_data.csv", index=False, encoding="utf-8-sig")



    print("\n 작업이 완료되었습니다.")
    print(


        f"피싱 데이터가 'phishing_data.csv' 파일로 저장되었습니다. (총 {len(phishing_final_df)}개 행)"
    )
    print(


        f"일반 데이터가 'normal_data.csv' 파일로 저장되었습니다. (총 {len(normal_final_df)}개 행)"
    )



except FileNotFoundError:


    print(f"\n 오류: '{file_path}' 경로에 파일이 없습니다.")


except IndexError:
    print(


        f"\n 오류: 'PHISHING_ROW_INDICES'에 지정된 행 번호가 실제 파일의 전체 행 수를 벗어났습니다."
    )


except KeyError as e:


    print(f"\n 오류: '{e}' 컬럼을 찾을 수 없습니다. 원본 파일에 해당 컬럼이 있는지 확인해주세요.")


except Exception as e:


    print(f"오류가 발생했습니다: {e}")

파일을 다음 경로에서 찾습니다: c:\Users\user\Downloads\woogawooga\woogawooga_project\dataset_create\github_dataset.csv

✅ 'github_dataset.csv' 파일을 성공적으로 불러왔습니다. (총 2927개 행)

 작업이 완료되었습니다.
피싱 데이터가 'phishing_data.csv' 파일로 저장되었습니다. (총 695개 행)
일반 데이터가 'normal_data.csv' 파일로 저장되었습니다. (총 2232개 행)


In [ ]:
# ==================================================================================
# Pandas와 정규표현식을 이용한 텍스트 데이터 문장 단위 분리 스크립트
# ==================================================================================
#
# [ 주요 기능 ]
# 1. 문장 분리: 'text' 열에 있는 긴 텍스트를 마침표(.), 물음표(?), 느낌표(!)를 기준으로
#    잘라내어 여러 개의 문장으로 만듭니다.
#    - 정규표현식(re.split)을 사용하여 분리하며, 각 문장이 원래의 구두점을 유지하도록 처리합니다.
#
# 2. 데이터 재구성 (Explode): 분리된 문장 리스트를 'explode' 함수를 사용해 각각의 행으로
#    만들어, '하나의 행 = 하나의 문장' 구조로 데이터를 변환합니다.
#
# 3. 데이터 정제:
#    - 원본 'text' 열을 삭제하고, 문장이 담긴 새 열의 이름을 'text'로 변경합니다.
#    - 분리 과정에서 생길 수 있는 빈 행이나 공백만 있는 행을 제거합니다.
#
# 4. 파일 저장: 최종적으로 정제된 데이터를 'sentences_data.csv' 파일로 저장하여,
#    문장 단위 분석을 위한 데이터셋을 생성합니다.
#
# [ 사용 방법 ]
# 1. 이 스크립트를 'normal_data.csv' 파일과 같은 폴더에 위치시킵니다.
# 2. 스크립트를 실행하면, 문장 분리 작업이 완료되고 결과 미리보기와 함께
#    'sentences_data.csv' 파일이 생성됩니다.
#
# ==================================================================================


import pandas as pd

import re


try:

    df = pd.read_csv("normal_data.csv")

except FileNotFoundError:

    print("오류: 'normal_data.csv' 파일을 찾을 수 없습니다.")

    df = pd.DataFrame()


if not df.empty:

    df["sentences"] = df["text"].apply(

        lambda x: [

            s.strip() + d

            for s, d in zip(

                re.split(r"([.?!])", str(x))[:-1:2], re.split(r"([.?!])", str(x))[1::2]
            )

        ]
    )


    df_exploded = df.explode("sentences")

    df_final = df_exploded.drop(columns=["text"]).rename(columns={"sentences": "text"})

    df_final = df_final[["file_name", "text"]]

    df_final = df_final.dropna(subset=["text"])

    df_final = df_final[df_final["text"].str.strip() != ""]


    print("✅ 작업 완료! 문장 분리 결과 (상위 10개):")

    print(df_final.head(10))


    df_final.to_csv('sentences_data.csv', index=False, encoding='utf-8-sig')

    print("\n'sentences_data.csv' 파일로 저장되었습니다.")

✅ 작업 완료! 문장 분리 결과 (상위 10개):
   file_name                                               text
0          0                           저는 여행 다니는 것을 굉장히 좋아하는데요.
0          0  그래가지고 스페인이나 뭐 영국 유럽 아니면 국내에서도 뭐 강릉이나 전주 같은 데를 ...
0          0        저 여행 다니는 거 되게 좋아해서 대학교 내내 여행을 엄청 많이 다녔었는데요.
0          0  제가 고등학교 때는 여행에 대해 흥미가 없었는데 그게 좀 아버지가 짠대로 패키지처럼...
0          0  그래서 대학교 간 이후로는 해외여행을 되게 많이 갔었는데 그중에서 제일 기 좋았던 ...
0          0                어~ 혹시 포르투갈이나 스페인 유럽 쪽 다녀오신 적 있으신가요?
0          0                                              어~ 네.
0          0                        저도 우연히 스페인과 포르투갈을 다녀왔었었습니다.
0          0  어~ 저는 스페인 중에서도 마드리드에 근교에 있었던 톨레도라는 지역이 굉장히 좋았는데요.
0          0  그 톨레도에서 특히 기억에 남았던 거는 거기에 대성당이 있는데 그 성당이 엄청 화려...

'sentences_data.csv' 파일로 저장되었습니다.
